<a href="https://colab.research.google.com/github/vanshkapil/sundayposts/blob/master/_notebooks/2020_10_28_Shopify_GraphQL_Queries_Getting_Started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Some Shopify GraphQL queries to get you started
Noob way

toc:true
branch: master
badges: true
comments: true
author: Vansh Kapil
categories: [shopify]

I have been trying to connect my AI shopping bot with Shopify API. This got me introduced to GraphQL. Here are some of the queries that I tried to get started. 

# API call function
We call this function with query as parameters to get the json object as response from Shopify. 

In [ ]:
def gq_query(data, variables):
    """
    Queries the Shopify
    :param data:
    :param variables:
    :return:
    """
    out = []  # the final output list of dictionaries to be converted to a DF
    shop = '<Your Store name>'
    token = '<Your store password>'
    headers = {
        'Content-Type': 'application/json',
        'X-Shopify-Access-Token': f'{token}',
    }

    response = requests.post(f'https://{shop}.myshopify.com/admin/api/2020-10/graphql.json',
                             json={'query': data, 'variables': variables}, headers=headers)
    return response.json()

#Getting product count for a collection.
You may note that I am formatting the query as string and not passing the variables in query variable. 
This is not the best practice, but it got me started. 

In [ ]:
def get_productcount_fromCollection(handle):
    """
    returns product count for given collection
    :param handle:
    :return: int count
    """
    data = '''
           {{
              collectionByHandle(handle: "{}"){{
                productsCount

              }}
            }}'''.format(handle)
    variable = ''' '''
    return gq_query(data, variable).get('data').get('collectionByHandle').get('productsCount')

# Getting products from a collection

This query is tries to fetch upto 50 products from the given collection handle. You can change the number of products and even create it as a variable. Be mindfull of the query cost. 
This works well if you have fewer products in a collection.

In [ ]:
def get_products_fromCollection(handle):
    """
    Get all the products for given collection
    :param handle:
    :return: List of dicts with title:title and price:price
    """
    
    obj = []
    num = get_productcount_fromCollection(handle)
    

    data = f'''
           query {{
              collectionByHandle(handle: "{handle}"){{
                products(first: 50){{
                   pageInfo {{
      hasNextPage
      hasPreviousPage
    }}
                  edges{{
                    cursor
                    node{{
                      id  
                      updatedAt
                      title
                      featuredImage{{
            transformedSrc
          }}  
                      priceRangeV2{{
                        minVariantPrice{{
                          currencyCode
                          amount
                        }}

                      }}
                   
                    }}

                  }}
                }}

              }}
            }}'''
    variable = ''' '''
    data_dict = gq_query(data, variable)
    
    

But what if there are thousands of products and you cannot fetch them in 1 call. That where we make use of *pageInfo and cursor* . 

In [ ]:
    while data_dict.get('data').get('collectionByHandle').get('products').get('pageInfo').get('hasNextPage'):
        time.sleep(5)
   
        cursor = data_dict.get('data').get('collectionByHandle').get('products').get('edges')[-1].get('cursor')
        data = f'''
                   query {{
                      collectionByHandle(handle: "{handle}"){{
                        products(first: 50, after: "{cursor}"){{
                           pageInfo {{
              hasNextPage
              hasPreviousPage
            }}
                          edges{{
                            cursor
                            node{{
                              id  
                              updatedAt
                              title
                              featuredImage{{
                    transformedSrc
                  }}  
                              priceRangeV2{{
                                minVariantPrice{{
                                  currencyCode
                                  amount
                                }}

                              }}

                            }}

                          }}
                        }}

                      }}
                    }}'''
        variable = ''' '''
        data_dict = gq_query(data, variable)

You may notice an attribute *pageInfo*  in the query which has a field called *hasNextPage* . hasNextPage gives out Boolean result (True or False). If its True it means there another set of products waiting on the next page

In [ ]:
data_dict.get('data').get('collectionByHandle').get('products').get('pageInfo').get('hasNextPage') 

Once we recognize that there is another page of products we need to move that page without re-reading the current batch of products. The current batch of products is recognized by something called *cursor*

In [ ]:
cursor = data_dict.get('data').get('collectionByHandle').get('products').get('edges')[-1].get('cursor')

The above call pulls the cursor of last product fetch, so we know that we have to fetch products after this cursor.

In [ ]:
query {{
                      collectionByHandle(handle: "{handle}"){{
                        products(first: 50, after: "{cursor}"){{
                           pageInfo {{
              hasNextPage
              hasPreviousPage
            }}

products(first: 50, after: "{cursor}") Asks for 50 products after the given cursor

#Fetch List of Collections

This function returns upto 200 collection's handles as a list

In [ ]:
def get_collections_list():
    """
    gets handles of all the collections of the store.
    :return: list of handles
    """

    data = f'''
                {{
                
                collections(first: 200){{
                  edges{{
                    node{{
                      handle
                    }}
                  }}
                }}
                  
                }}
                
                '''
    variable = ''' '''
    data_dict = gq_query(data, variable)
    handle_list=[]
    for n in range(len(data_dict['data'].get('collections').get('edges'))):
        handle_list.append(data_dict['data'].get('collections').get('edges')[n].get('node').get('handle'))
    return handle_list

# Get product Updation details
There could be a situation wherein you may want to check when a product was last updated/modified or you may want to fetch products updated after a certain timestamp. This can be accomplished by looking for updatedAt attribute.

In [ ]:
def get_updated_products():
    """
    gets updated products based on updateAt timestamp from shopify API
    :return:
    """


    data = f'''
               {{
                
                  products(query:"updated_at:>{last_update_timestamp}" first:800) {{
                    edges {{
                      node {{
                         id
                            title
                            updatedAt
                            featuredImage{{
                              transformedSrc
                            }}
                            priceRangeV2{{
                              minVariantPrice{{
                                currencyCode
                                amount
                              }}
                            }}
                        
                      }}
                    }}
                  }}
                  
                }}
            '''
    variable = ''' '''
    data_dict = gq_query(data, variable)

last_update_timestamp is the variable for timestamp

Thats it for now, I will update this post as an when I use more queries in my project